In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

In [3]:
# z ~ N(0, I)
# x[0:d] = z[0:d]
# x[d:D] = z[d:D] * scale(z[0:d]) + loc(z[z[0:d]])

The realNVP architecture is a special case of the autoregressive flow. The data is represented by the D dimensional vector x. We will partioned the vector x in two: the first d elements of x are set to be equal to the first d elements of z. This d elements of z are also inputs to a function or functions that produce mean and standard deviation parameters for the Gaussian distribution that model the remaining features in the vector x. The scale and loc functions can be any functions.

In [4]:
shift_and_log_scale_fn = tfb.real_nvp_default_template(
    hidden_layers = [32, 32], activation=tf.nn.relu)

As with the masked autoregressive flow bijector, there is a default option that you can use for this function. This is called the `real_nvp_default_template`. This is function that wraps an implementation of a feed forward neural network, and you can set the number of hidden layers and activation function. The relu activation is the default for this implementation. So the `shift_and_log_scale_fn` is now the feed forward neural network function. 

In [6]:
shift_and_log_scale_fn(tf.random.normal([2]), 1)

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.09373578], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.09835492], dtype=float32)>)

The function takes two arguments, the first is the network input which above we make one of length 2. In other words d is equal to 2. The second argument is the size of the output, above equal to 1. This means that the event_size D is equal to 3. The output of the network is a length 2 tuple, containing the mean and scale parameters for the equation presented in the beginning of the document. 

In [7]:
# x[d:D] = z[d:D] * scale(z[0:d]) + loc(z[z[0:d]])

For this example, the equation above is of length one. The `real_nvp_default_template` is create in a way that the network is created the first time that the function is called. Subsequent calls would need to have the same size arguments to it, otherwise it would raise an error. 

In [8]:
shift_and_log_scale_fn = tfb.real_nvp_default_template(
    hidden_layers = [32, 32], 
    activation=tf.nn.relu,
    shift_only=True)

We can also use the `shift_only` keyword argument, which makes the scale parameter set to 1.

In [9]:
shift_and_log_scale_fn(tf.random.normal([2]), 1)

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.23823035], dtype=float32)>,
 None)

This way the function still returns a shape 2 tuple but the second element is none.

This shift only version of realNVP is precisely the NICE model that realNVP builds on. NICE stands for Non-linear Independent Components Estimation. In this shift only case, the Jacobian is the identity matrix, which means that the bijective transformation is volume preserving. RealNVP stands for Real Value Non-Volume Preserving and the main difference is the introduction of the scaling parameter. It also means that the Jacobian determinant need not to be the identity. So the transformation is not volume preserving. 

In [10]:
realnvp_bijector = tfb.RealNVP(
    num_masked=2, shift_and_log_scale_fn=shift_and_log_scale_fn)

Now, we can define the realNVP bijector. The RealNVP class, which is in the bijectors module, has a `shift_and_log_scale_fn` keyword argument. Here we are passing the realNVP default template. We are also passing the `num_masked` argument and setting it to 2. This number corresponds to d. It defines the first partition of the vectors x and z. 

In [11]:
def forward(z):
    x = tf.zeros_like(z)
    x[0:d] = z[0:d]
    shift, log_scale = shift_and_log_scale_fn(z[0:d])
    x[d:D] = z[d:D] * tf.math.exp(log_scale) + shift
    return x

Above we have the pseudocode for the realNVP bijector forward transformation. We start by initializing the vector x and we set the first d elements to be the same as the first d elements in z. This first d elements are then used to generate the mean and log scale parameters that define the Gaussian distribution for the remaining features.

In [12]:
def inverse(x):
    z = tf.zeros_like(x)
    z[0:d] = x[0:d]
    shift, log_scale = shift_and_log_scale_fn(x[0:d])
    z[d:D] = (x[d:D]-shift) * tf.math.exp(-log_scale)
    return z

The equations can easily be inverted to generate the inverse transformation of the realNVP bijector. For the realNVP, the forward and inverse transformations are actually very similar computationally. This is not the case for the other autoregressive transformations that we saw. The trade off is that the model is not as expressive as the masked autoregressive flow. 

In [13]:
mvn = tfd.MultivariateNormalDiag(loc=[0.,0.,0.])
realnvp = tfd.TransformedDistribution(distribution=mvn, bijector=realnvp_bijector)

Now that we have our bijector we can use it to create our transformed distribution. Here we are defining a multivariate normal diaginal distribution with event shape equal to 3. Notice that the event shape of the base distribution is the one required by the bijector that we are using. 

In [14]:
realnvp_bijector = tfb.RealNVP(
    fraction_masked=0.5, shift_and_log_scale_fn=shift_and_log_scale_fn)

Instead of the defining explicitly the number of units to mask, we can use the fraction_masked keyword.

In [15]:
permute = tfb.Permute(permutation=[1,2,0])

realnvp1 = tfb.RealNVP(fraction_masked=0.5,
                      shift_and_log_scale_fn=tfb.real_nvp_default_template(hidden_layers=[32, 32]))
realnvp2 = tfb.RealNVP(fraction_masked=0.5,
                      shift_and_log_scale_fn=tfb.real_nvp_default_template(hidden_layers=[32, 32]))
realnvp3 = tfb.RealNVP(fraction_masked=0.5,
                      shift_and_log_scale_fn=tfb.real_nvp_default_template(hidden_layers=[32, 32]))

chained_bijector = tfb.Chain([realnvp3, permute, realnvp2, permute, realnvp1])

As we saw previously, the realNVP bijector leaves part of the input vector unchanged, so we need to combine multiple realNVP layers together to produce a bijector that can transform all the vector components. In this example we are creating a transformed distribution that uses a bijector consisting of 3 realNVP layers. We are creating the realNVP layers all in the same way, by masking half of the input vector and using a feed forward network with two hidden layers. We are also creating a permute bijector, that changes the order of the features in the vector. We are then chaining these 3 realNVP layers together with the permute bijector in between each one. This has the overall effect of mixing the vector components so that all components can be transformed.